In [68]:
import os
import re
from dotenv import load_dotenv
from urllib.parse import urlparse, parse_qs
from googleapiclient.discovery import build
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [69]:
load_dotenv()
yt_api_key = os.getenv("YOUTUBE_API_KEY")
spotify_client_id=os.getenv("SPOTIFY_CLIENT_ID")
spotify_client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
youtube = build('youtube', 'v3', developerKey=yt_api_key)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=spotify_client_id,
                                               client_secret=spotify_client_secret,
                                               redirect_uri="http://127.0.0.1:8888/callback",
                                               scope="user-library-read"))

Get Youtube Related Statistics

In [82]:
def get_youtube_data(video_url):
    video_id = video_url.split("v=")[1]
    video_request = youtube.videos().list(part="snippet,statistics", id=video_id)
    video_response = video_request.execute()
    
    if not video_response['items']:
        return None
        
    video_item = video_response['items'][0]

    artist_name = video_item['snippet']['channelTitle'].replace(" - Topic", "")
    search_request = youtube.search().list(
        q=f"{artist_name} official", 
        type="channel", 
        part="id", 
        maxResults=1
    )
    search_response = search_request.execute()
    
    if not search_response['items']:
        official_channel_id = video_item['snippet']['channelId']
    else:
        official_channel_id = search_response['items'][0]['id']['channelId']    
    
    channel_request = youtube.channels().list(part="snippet,statistics", id=official_channel_id)
    channel_response = channel_request.execute()
    channel_item = channel_response['items'][0]

    return {
        "Video ID": video_id,
        "Title": video_item['snippet']['title'],
        "Channel Name": channel_item['snippet']['title'],
        "Upload Date": video_item['snippet']['publishedAt'],
        "View Count": video_item['statistics'].get('viewCount', 0),
        "Likes": video_item['statistics'].get('likeCount', 0),
        "Comments": video_item['statistics'].get('commentCount', 0),
        "Tags": video_item['snippet'].get('tags', []),
        "Category ID": video_item['snippet']['categoryId'],
        "Channel Subscriber Count": channel_item['statistics']['subscriberCount'],
        "Channel Total View Count": channel_item['statistics']['viewCount'],
        # "Channel Video Count": channel_item['statistics']['videoCount'],
        # "Channel Age (Created At)": channel_item['snippet']['publishedAt']
    }

In [92]:
def get_spotify_data(title, artist):
    lookup = f"{title} {artist}"
    search_result = sp.search(q=lookup, type="track", limit=1)

    if not search_result['tracks']['items']:
        return {}

    track = search_result['tracks']['items'][0]
    artist_id = track['artists'][0]['id']
    artist_data = sp.artist(artist_id)

    return {
        "spotify_track_id": track['id'],
        "spotify_track_name": track['name'],
        "spotify_track_popularity": track['popularity'],
        "spotify_artist_name": artist_data['name'],
        "spotify_artist_followers": artist_data['followers']['total'],
        "spotify_artist_genres": artist_data['genres'],
        "spotify_release_date": track['album']['release_date']
    }

In [93]:
def get_combined_data(video_url):
    youtube_data = get_youtube_data(video_url)
    if youtube_data is None:
        return None

    sp_data = get_spotify_data(youtube_data["Title"], youtube_data["Channel Name"])

    if sp_data is None:
        return None

    return {**youtube_data, **sp_data}

In [94]:
video_url = "https://www.youtube.com/watch?v=Bf01riuiJWA"
data_full = get_combined_data(video_url)
for k, v in data_full.items():
    print(f"{k}: {v}")

Video ID: Bf01riuiJWA
Title: Exit Music (For A Film)
Channel Name: Radiohead
Upload Date: 2016-04-06T21:57:35Z
View Count: 40541291
Likes: 555344
Comments: 8603
Tags: ['Radiohead', 'OK Computer', 'Exit Music (For A Film)']
Category ID: 10
Channel Subscriber Count: 4950000
Channel Total View Count: 4379860343
spotify_track_id: 0z1o5L7HJx562xZSATcIpY
spotify_track_name: Exit Music (For A Film)
spotify_track_popularity: 81
spotify_artist_name: Radiohead
spotify_artist_followers: 14560375
spotify_artist_genres: ['art rock', 'alternative rock']
spotify_release_date: 1997-05-28
